<a href="https://colab.research.google.com/github/quimHM/QHM_TFG_repository/blob/main/MAB_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://towardsdatascience.com/multi-armed-bandits-and-reinforcement-learning-dc9001dcb8da
#https://towardsdatascience.com/the-upper-confidence-bound-ucb-bandit-algorithm-c05c2bf4c13f

#https://github.com/WhatIThinkAbout/BabyRobot/tree/master/Multi_Armed_Bandits

In [2]:
# import modules 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import math
import random
%matplotlib inline

In [3]:
class PowerSocket:
    """ the base power socket class """
    
    def __init__(self, q, cl, var=1):                
        self.q = q        # the true reward value 
        self.confidence_level = cl          
        self.var = var   
        self.initialize() # reset the socket
        
    def initialize(self):
        self.Q = 0   # the estimate of this socket's reward value                
        self.n = 0   # the number of times this socket has been tried        
    
    def charge(self):
        """ return a random amount of charge """
        
        # the reward is a guassian distribution with unit variance around the true
        # value 'q'
        value = self.var * np.random.randn() + self.q        
        
        # never allow a charge less than 0 to be returned        
        return 0 if value < 0 else value
               
    def update(self,R):
        """ update this socket after it has returned reward value 'R' """     
    
        # increment the number of times this socket has been tried
        self.n += 1

        # the new estimate of the mean is calculated from the old estimate
        self.Q = (1 - 1.0/self.n) * self.Q + (1.0/self.n) * R

    def uncertainty(self, t): 
        """ calculate the uncertainty in the estimate of this socket's mean """
        if self.n == 0: return float('inf')                         
        return self.confidence_level * (np.sqrt(np.log(t) / self.n))         
        
    def sample(self,t):
        """ the UCB reward is the estimate of the mean reward plus its uncertainty """
        #print(self.uncertainty(t))
        return self.Q + self.uncertainty(t) 

In [66]:
# return the index of the largest value in the supplied list
# - arbitrarily select between the largest values in the case of a tie
# (the standard np.argmax just chooses the first value in the case of a tie)
def random_argmax(value_list):
  """ a random tie-breaking argmax"""
  values = np.asarray(value_list)
  return np.argmax(np.random.random(values.shape) * (values==values.max()))

# return the index of the smallest value in the supplied list
# - arbitrarily select between the smallest values in the case of a tie
# (the standard np.argmin just chooses the first value in the case of a tie)
def random_argmin(value_list):
  """ a random tie-breaking argmin"""
  values = np.asarray(value_list)
  return np.argmin(np.random.random(values.shape) * (values==values.min()))


In [5]:
class SocketTester():
    """ create and test a set of sockets over a single test run """

    def __init__(self, socket, socket_order, socket_vars, confidence_level):  
        
        # create supplied socket type with a mean value defined by socket order 
        # self.sockets = [socket(q, confidence_level) for q in socket_order]     
        self.sockets = [socket(socket_order[s], confidence_level, socket_vars[s]) for s in range(len(socket_order))]

        # set the number of sockets equal to the number created
        self.number_of_sockets = len(self.sockets)

        self.number_of_stats = 2                 
            
    def initialize_run(self, number_of_steps):
        """ reset counters at the start of a run """
        
        # save the number of steps over which the run will take place
        self.number_of_steps = number_of_steps
        
        # reset the actual number of steps that the test ran for
        self.total_steps = 0
        
        # monitor the total reward obtained over the run
        self.total_reward = 0
        
        # the current total reward at each timestep of the run
        self.total_reward_per_timestep = []
        
        # the actual reward obtained at each timestep
        self.reward_per_timestep = []
           
        # stats for each time-step
        # - by default records: estimate, number of trials
        self.socket_stats = np.zeros(shape=(number_of_steps+1, 
                                            self.number_of_sockets, 
                                            self.number_of_stats))
        
        # ensure that all sockets are re-initialized
        for socket in self.sockets: socket.initialize()
            
                                
    def charge_and_update(self,socket_index):
        """ charge from & update the specified socket and associated parameters """
        
        # charge from the chosen socket and update its mean reward value
        reward = self.sockets[socket_index].charge()
        self.sockets[socket_index].update(reward)

        # update the total reward
        self.total_reward += reward   
        
        # store the current total reward at this timestep
        self.total_reward_per_timestep.append(self.total_reward)
        
        # store the reward obtained at this timestep
        self.reward_per_timestep.append(reward)        
        
        
    def get_socket_stats( self, t ):
        """ get the current information from each socket """        
        socket_stats = [[socket.Q, socket.n] for socket in self.sockets]
        return socket_stats     
    
    def get_mean_reward( self ):
        """ the total reward averaged over the number of time steps """
        return (self.total_reward/self.total_steps)
    
    def get_total_reward_per_timestep( self ):
        """ the cumulative total reward at each timestep of the run """
        return self.total_reward_per_timestep
    
    def get_reward_per_timestep( self ):
        """ the actual reward obtained at each timestep of the run """
        return self.reward_per_timestep
    
    def get_estimates(self):
        """ get the estimate of each socket's reward at each timestep of the run """
        return self.socket_stats[:,:,0]  
    
    def get_number_of_trials(self):
        """ get the number of trials of each socket at each timestep of the run """
        return self.socket_stats[:,:,1]          
                
    def get_socket_percentages( self ):
        """ get the percentage of times each socket was tried over the run """
        return (self.socket_stats[:,:,1][self.total_steps]/self.total_steps)        
    
    def get_time_steps( self ):
        """ get the number of time steps that the test ran for """
        return self.total_steps
    
    def select_socket( self, t ):
        """ Greedy Socket Selection"""
        
        # choose the socket with the current highest mean reward or arbitrarily
        # select a socket in the case of a tie            
        socket_index = random_argmax([socket.sample(t+1) for socket in self.sockets]) 
        return socket_index     
    
    
    def run( self, number_of_steps, maximum_total_reward = float('inf')):  
        """ perform a single run, over the set of sockets, 
            for the defined number of steps """
        
        # reset the run counters
        self.initialize_run(number_of_steps)
        
        # loop for the specified number of time-steps
        for t in range(number_of_steps):

            # get information about all sockets at the start of the time step
            self.socket_stats[t] = self.get_socket_stats(t)            
            
            # select a socket
            socket_index = self.select_socket(t)
            
            # charge from the chosen socket and update its mean reward value
            self.charge_and_update(socket_index)
            
            # test if the accumulated total reward is greater than the maximum
            if self.total_reward > maximum_total_reward:
                break
       
        # save the actual number of steps that have been run
        self.total_steps = t    
    
        # get the stats for each socket at the end of the run        
        self.socket_stats[t+1] = self.get_socket_stats(t+1)           
        
        return self.total_steps, self.total_reward
  

In [6]:
def mus_and_devs_random():
  devs = []
  mus = []
  for i in range(8):
    d = round(random.uniform(5.0,15.0),2)
    m = round(random.uniform(70.0,100.0),2)
    devs.append(d)
    mus.append(m)
  return mus,devs

In [7]:
def mus_and_devs():
  devs = [10.0]*8
  mus = []
  for i in range(8):
    m = 100.0-i*5.0
    mus.append(m)
  return mus,devs

In [8]:
#mus,devs = mus_and_devs_random()
mus,devs = mus_and_devs()

#devs = [9.3, 5.67, 11.93, 9.18, 13.66, 8.94, 6.83, 7.91]
#mus = [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]

print(devs)
print(mus)
test = SocketTester(PowerSocket, mus, devs, 5)
test.run(1000)
test.get_number_of_trials()[-1]

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
[100.0, 95.0, 90.0, 85.0, 80.0, 75.0, 70.0, 65.0]


array([963.,  27.,   3.,   3.,   1.,   1.,   1.,   1.])

In [53]:
class SocketTesterBatch():
    """ create and test a set of sockets over a single test run """

    def __init__(self, socket, socket_order, socket_vars, confidence_level):  
        
        # create supplied socket type with a mean value defined by socket order 
        self.sockets = [socket(socket_order[s], confidence_level, socket_vars[s]) for s in range(len(socket_order))]
        
        # set the number of sockets equal to the number created
        self.number_of_sockets = len(self.sockets)

        self.number_of_stats = 2                 
            
    def initialize_run(self, number_of_steps):
        """ reset counters at the start of a run """
        
        # save the number of steps over which the run will take place
        self.number_of_steps = number_of_steps
        
        # reset the actual number of steps that the test ran for
        self.total_steps = 0
        
        # monitor the total reward obtained over the run
        self.total_reward = 0
        
        # the current total reward at each timestep of the run
        self.total_reward_per_timestep = []
        
        # the actual reward obtained at each timestep
        self.reward_per_timestep = []
           
        # stats for each time-step
        # - by default records: estimate, number of trials
        self.socket_stats = np.zeros(shape=(number_of_steps+1, 
                                            self.number_of_sockets, 
                                            self.number_of_stats))
        
        # ensure that all sockets are re-initialized
        for socket in self.sockets: socket.initialize()
            
                                
    def charge_and_update(self,socket_index):
        """ charge from & update the specified socket and associated parameters """
        
        # charge from the chosen socket and update its mean reward value
        reward = self.sockets[socket_index].charge()
        self.sockets[socket_index].update(reward)

        # update the total reward
        self.total_reward += reward   
        
        # store the current total reward at this timestep
        self.total_reward_per_timestep.append(self.total_reward)
        
        # store the reward obtained at this timestep
        self.reward_per_timestep.append(reward)        
        
        
    def get_socket_stats( self, t ):
        """ get the current information from each socket """        
        socket_stats = [[socket.Q, socket.n] for socket in self.sockets]
        return socket_stats     
    
    def get_mean_reward( self ):
        """ the total reward averaged over the number of time steps """
        return (self.total_reward/self.total_steps)
    
    def get_total_reward_per_timestep( self ):
        """ the cumulative total reward at each timestep of the run """
        return self.total_reward_per_timestep
    
    def get_reward_per_timestep( self ):
        """ the actual reward obtained at each timestep of the run """
        return self.reward_per_timestep
    
    def get_estimates(self):
        """ get the estimate of each socket's reward at each timestep of the run """
        return self.socket_stats[:,:,0]  
    
    def get_number_of_trials(self):
        """ get the number of trials of each socket at each timestep of the run """
        return self.socket_stats[:,:,1]          
                
    def get_socket_percentages( self ):
        """ get the percentage of times each socket was tried over the run """
        return (self.socket_stats[:,:,1][self.total_steps]/self.total_steps)        
    
    def get_time_steps( self ):
        """ get the number of time steps that the test ran for """
        return self.total_steps
    
    def select_socket( self, t ):
        """ Greedy Socket Selection"""
        
        # choose the socket with the current highest mean reward or arbitrarily
        # select a socket in the case of a tie            
        mask = [self.get_number_of_trials()[t][i]<self.possibles[i] for i in range(len(self.sockets))]
        #print(mask)
        available = [self.sockets[i] for i in range(len(self.sockets)) if (mask[i])]
        socket_max = random_argmax([socket.sample(t+1) for socket in available]) 
        #print(socket_max)
        socket_index = self.sockets.index(available[socket_max])
        return socket_index     
    
    def return_increments(self, decisions_to_consider):
        presence = np.array([i/len(decisions_to_consider) for i in self.possibles])
        selected_presence = np.array(self.get_socket_percentages())
        diff = np.subtract(selected_presence, presence)
        diff = np.divide(diff,presence, out=np.zeros_like(selected_presence), where=presence!=0)
        #print(diff)
        #print(diff.sum())
        return diff
    
    def run( self, decisions_to_consider, max_percent_decisions=1, prints_bool = True):  
        """ perform a single run, over the set of sockets, 
            for the defined number of steps """
        
        # reset the run counters
        self.initialize_run(len(decisions_to_consider))

        self.possibles = [0]*8 #TODO: DEPENDS ON N OF CLUSTER (DECISIONS.UNIQUE() ISNT ENOUGH BECAUSE IT CAN LACK SOME INSTANCES)
        for i in np.unique(np.array(decisions_to_consider)):
          self.possibles[i] = decisions_to_consider.count(i) 

        usos = 0
        usos_maxims = math.floor(max_percent_decisions*len(decisions_to_consider))
        #print(usos_maxims)

        # loop for the specified number of time-steps
        for t in range(len(decisions_to_consider)):

            # get information about all sockets at the start of the time step
            self.socket_stats[t] = self.get_socket_stats(t)            
            
            # select a socket
            socket_index = self.select_socket(t)
            #if(decisions_to_consider[t]==socket_index):
            if(self.get_number_of_trials()[t][socket_index]<self.possibles[socket_index]):
              # charge from the chosen socket and update its mean reward value
              self.charge_and_update(socket_index)
              usos+=1
              
              if usos > usos_maxims:
                  #print(usos,usos_maxims)
                  break
        # save the actual number of steps that have been run
        self.total_steps = t    
    
        # get the stats for each socket at the end of the run        
        self.socket_stats[t+1] = self.get_socket_stats(t+1)           
        
        if(prints_bool):
          print("Reward distribution:",[s.q for s in self.sockets])
          #print("Intances of each cluster:",self.possibles)
          print("Presence of each in batch (%):",[100*i/len(decisions_to_consider) for i in self.possibles])
          #print("Times selected:",self.get_number_of_trials()[t])
          print("Relative approval rate (%):",[round(100*self.get_number_of_trials()[t][i]/self.possibles[i],2) if self.possibles[i]>0.0 else 0.0 for i in range(len(self.possibles))])
          print("Percentage over total selected (%):",[round(100*p,2) for p in self.get_socket_percentages()])
          #print("For a total reward:",self.total_reward)


        return self.total_steps, self.total_reward, self.return_increments(decisions_to_consider)
  

In [54]:
devs = [9.3, 5.67, 11.93, 9.18, 13.66, 8.94, 6.83, 7.91]
mus = [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
#mus,devs = mus_and_devs()
test2 = SocketTesterBatch(PowerSocket, mus, devs, 10)

decisions = random.choices(range(8), k=1000)
decisions = []
for i in range(8):
  for j in range(125):
    decisions.append(i)
#decisions = [[125]]*8
ts,tr,_ = test2.run(decisions)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5]
Relative approval rate (%): [100.0, 100.0, 100.0, 100.0, 100.0, 99.2, 100.0, 100.0]
Percentage over total selected (%): [12.51, 12.51, 12.51, 12.51, 12.51, 12.41, 12.51, 12.51]


In [11]:
ts,tr,_ = test2.run(decisions, 0.5)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5, 12.5]
Relative approval rate (%): [100.0, 100.0, 1.6, 2.4, 35.2, 0.8, 100.0, 60.0]
Percentage over total selected (%): [25.0, 25.0, 0.4, 0.6, 8.8, 0.2, 25.0, 15.0]


In [13]:
#sample_bias = [0.555, 0.24, 0.023, 0.011, 0.087, 0.081, 0.001, 0.001]
#devs = [9.3, 5.67, 11.93, 9.18, 13.66, 8.94, 6.83, 7.91]
#mus = [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
weights_m = []
for m in mus:
  weights_m.append((m*m*m*m*m*m*m*m*m)/1000*(m*m*m*m*m*m*m*m*m)/1000)
sample_bias_decisions = random.choices(range(8),weights=weights_m,k=1000)
print(sample_bias_decisions)
ts,tr,_ = test2.run(sample_bias_decisions, 0.8)

[7, 4, 6, 1, 1, 6, 0, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 1, 1, 7, 6, 6, 1, 1, 2, 2, 1, 6, 1, 1, 4, 1, 6, 1, 6, 1, 1, 4, 6, 6, 6, 7, 0, 6, 0, 1, 0, 2, 1, 6, 1, 1, 1, 6, 0, 0, 7, 0, 5, 7, 6, 1, 6, 6, 6, 7, 6, 3, 1, 0, 1, 1, 6, 6, 1, 6, 6, 2, 1, 7, 0, 6, 6, 6, 1, 0, 1, 2, 3, 7, 1, 1, 7, 1, 6, 2, 0, 1, 1, 1, 7, 7, 2, 1, 4, 7, 6, 0, 7, 1, 1, 6, 0, 1, 7, 6, 1, 6, 1, 4, 6, 7, 1, 1, 6, 1, 6, 0, 7, 2, 1, 2, 1, 7, 0, 6, 6, 1, 1, 4, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 6, 0, 6, 6, 6, 0, 1, 6, 1, 0, 6, 6, 6, 1, 2, 1, 7, 2, 0, 1, 0, 6, 4, 1, 4, 1, 0, 2, 0, 0, 0, 1, 7, 0, 6, 6, 1, 6, 7, 0, 1, 0, 0, 7, 1, 7, 4, 6, 1, 6, 2, 1, 6, 1, 6, 1, 6, 1, 3, 6, 1, 4, 7, 6, 1, 1, 7, 1, 1, 4, 1, 7, 3, 0, 0, 0, 1, 4, 4, 1, 7, 6, 1, 1, 0, 1, 6, 1, 6, 7, 6, 2, 6, 1, 1, 1, 1, 1, 7, 4, 0, 1, 1, 7, 6, 1, 4, 7, 1, 0, 0, 6, 4, 1, 6, 1, 0, 1, 0, 7, 7, 1, 0, 6, 1, 6, 0, 1, 1, 2, 7, 1, 6, 1, 3, 1, 1, 1, 6, 7, 4, 6, 1, 1, 6, 1, 2, 6, 6, 6, 6, 4, 1, 6, 2, 6, 6, 6, 7, 1, 1, 6, 7, 6, 1, 2, 1, 7, 4, 7, 1, 1, 7, 1, 6, 4, 1, 0, 0, 6, 4, 

In [14]:
a=np.array([9.7, 36.7, 5.6, 0.9, 7.2, 0.1, 29.3, 10.5])
#b=np.array([12.12, 45.88, 2.0, 1.12, 0.25, 0.12, 36.62, 1.88])
b=np.array([12.12, 45.88, 0.75, 0.25, 1.38, 0.12, 36.62, 2.88])


diff = np.subtract(b,a)
print(diff)
diff = np.divide(diff,a)
print(diff)

[ 2.42  9.18 -4.85 -0.65 -5.82  0.02  7.32 -7.62]
[ 0.24948454  0.25013624 -0.86607143 -0.72222222 -0.80833333  0.2
  0.24982935 -0.72571429]


###**NON-STATIC**

In [15]:
#def f(probs, val, ind):
#    probs[ind] += val
#    mask = np.ones(len(probs), bool)
#    mask[ind] = False
#    probs[mask] -= val / (len(probs) - 1)
#    return np.clip(probs, 0, np.infty) / np.sum(np.clip(probs, 0, np.infty))

In [55]:
def cummulative_sample(tester,iter,orig,coef):
  instances_distribution = np.array(orig)
  for i in range(iter):
    pbool=False
    if(i%math.ceil(iter/4)==0 or i==iter-1):
      print(i)
      #print(list(instances_distribution))
      pbool=True

    decisions = random.choices(range(8),weights=list(instances_distribution),k=5000)
    ts,tr,ti = tester.run(decisions, 0.8, pbool)
    
    #for i in range(len(ti)):
    #  instances_distribution = f(instances_distribution,ti[i],i)
    instances_distribution = np.add(instances_distribution,ti*coef*100)
    instances_distribution = np.clip(instances_distribution,0,None)
    instances_distribution = instances_distribution / instances_distribution.sum()

In [56]:
o_sample_bias = [0.555, 0.24, 0.023, 0.011, 0.087, 0.081, 0.001, 0.001]
#o_sample_bias = weights_m
cummulative_sample(test2,35,o_sample_bias,0.001)

0
Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [55.76, 24.44, 2.24, 1.2, 8.3, 7.86, 0.16, 0.04]
Relative approval rate (%): [97.85, 100.0, 20.54, 6.67, 2.65, 0.51, 100.0, 100.0]
Percentage over total selected (%): [68.2, 30.55, 0.57, 0.1, 0.27, 0.05, 0.2, 0.05]
9
Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [34.42, 41.7, 0.0, 0.0, 0.0, 0.0, 20.96, 2.92]
Relative approval rate (%): [41.89, 100.0, 0.0, 0.0, 0.0, 0.0, 100.0, 100.0]
Percentage over total selected (%): [18.02, 52.12, 0.0, 0.0, 0.0, 0.0, 26.2, 3.65]
18
Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [0.0, 81.72, 0.0, 0.0, 0.0, 0.0, 18.28, 0.0]
Relative approval rate (%): [0.0, 96.21, 0.0, 0.0, 0.0, 0.0, 7.55, 0.0]
Percentage over total selected (%): [0.0, 98.28, 0.0, 0.0, 0.0, 0.0, 1.72, 0.0]
27
Reward distribution: [92.1, 98.48, 88.49, 80.36

In [ ]:
no_sample_bias = [1/8]*8
cummulative_sample(test2,35,no_sample_bias,0.05)

In [ ]:
no_sample_bias = [1/8]*8
cummulative_sample(test2,150,no_sample_bias,0.05)

In [117]:
def nonstatic(iter, rewards_d_o_mu, rewards_d_o_dev, r_coef, instances_d_o, i_coef):
  rewards_d = np.array(rewards_d_o_mu)
  instances_d = np.array(instances_d_o)

  #iter = 1000
  for i in range(iter):
    pbool=False
    if(i%math.ceil(iter/4)==0 or i==iter-1):
        print(i)
        #print(list(instances_distribution))
        pbool=True

    updated_test = SocketTesterBatch(PowerSocket, list(rewards_d), rewards_d_o_dev, 10)
    decisions = random.choices(range(8),weights=list(instances_d),k=5000)
    ts,tr,ti = updated_test.run(decisions, 0.8, pbool)

    increments = ti
    #increments = np.subtract(np.array(updated_test.get_socket_percentages()),0.5)
    #print(increments)

    rewards_d = np.add(rewards_d,increments*r_coef*100)
    rewards_d = np.clip(rewards_d,0,100)
    #print(rewards_d)

    instances_d = np.add(instances_d,increments*i_coef)
    instances_d = np.clip(instances_d,0,None)
    instances_d = instances_d / instances_d.sum()

In [63]:
rewards_bias = np.array([.8555, .8193, .8284, .7994, .7103, .6635, .7471, .6731])*100
rewards_nonbias = [100/2]*8

instances_bias = [0.555, 0.24, 0.023, 0.011, 0.087, 0.081, 0.001, 0.001]
instances_nonbias = [1/8]*8

In [64]:
nonstatic(500, rewards_bias, 0.001, instances_bias, 0.001)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [56.8, 23.68, 2.2, 1.08, 8.38, 7.64, 0.16, 0.06]
Relative approval rate (%): [100.0, 78.97, 100.0, 100.0, 8.35, 3.93, 100.0, 100.0]
Percentage over total selected (%): [71.0, 23.38, 2.75, 1.35, 0.88, 0.38, 0.2, 0.08]
125
Reward distribution: [85.86249999999971, 81.91065562412322, 83.15249999999972, 80.25249999999971, 69.88230818120334, 65.16596967318351, 75.02249999999971, 67.58629694034595]
Presence of each in batch (%): [55.36, 24.8, 2.76, 1.58, 7.58, 7.16, 0.4, 0.36]
Relative approval rate (%): [100.0, 74.03, 100.0, 100.0, 10.82, 5.03, 100.0, 100.0]
Percentage over total selected (%): [69.2, 22.95, 3.45, 1.98, 1.03, 0.45, 0.5, 0.45]
250
Reward distribution: [86.17499999999943, 81.76273036679844, 83.46499999999943, 80.56499999999943, 68.73875820765618, 63.98368291739301, 75.33499999999943, 67.65884150605889]
Presence of each in batch (%): [57.4, 24.44, 3.14, 1.76, 6.2, 5.84,

In [ ]:
nonstatic(2500, rewards_nonbias, 0, instances_bias, 0.01)

In [ ]:
nonstatic(100, rewards_bias, 0, instances_nonbias, 0)

In [ ]:
nonstatic(100, rewards_bias, 0.1, instances_nonbias, 0)

In [ ]:
nonstatic(100, rewards_bias, 0, instances_bias, 0.01)

In [ ]:
nonstatic(100, rewards_bias, 0.01, instances_bias, 0)

In [ ]:
nonstatic(100, rewards_bias, 0.01, instances_bias, 0.01)

###**SOLUTION?**

In [2]:
class SocketTesterBatchSOLUTION():
    """ create and test a set of sockets over a single test run """

    def __init__(self, socket, socket_order, socket_vars, confidence_level, sub_opt):  
        
        self.subopt_coef = sub_opt

        # create supplied socket type with a mean value defined by socket order 
        self.sockets = [socket(socket_order[s], confidence_level, socket_vars[s]) for s in range(len(socket_order))]
        
        # set the number of sockets equal to the number created
        self.number_of_sockets = len(self.sockets)

        self.number_of_stats = 2                 
            
    def initialize_run(self, number_of_steps):
        """ reset counters at the start of a run """
        
        # save the number of steps over which the run will take place
        self.number_of_steps = number_of_steps
        
        # reset the actual number of steps that the test ran for
        self.total_steps = 0
        
        # monitor the total reward obtained over the run
        self.total_reward = 0
        
        # the current total reward at each timestep of the run
        self.total_reward_per_timestep = []
        
        # the actual reward obtained at each timestep
        self.reward_per_timestep = []
           
        # stats for each time-step
        # - by default records: estimate, number of trials
        self.socket_stats = np.zeros(shape=(number_of_steps+1, 
                                            self.number_of_sockets, 
                                            self.number_of_stats))
        
        # ensure that all sockets are re-initialized
        for socket in self.sockets: socket.initialize()
            
                                
    def charge_and_update(self,socket_index):
        """ charge from & update the specified socket and associated parameters """
        
        # charge from the chosen socket and update its mean reward value
        reward = self.sockets[socket_index].charge()
        self.sockets[socket_index].update(reward)

        # update the total reward
        self.total_reward += reward   
        
        # store the current total reward at this timestep
        self.total_reward_per_timestep.append(self.total_reward)
        
        # store the reward obtained at this timestep
        self.reward_per_timestep.append(reward)        
        
        
    def get_socket_stats( self, t ):
        """ get the current information from each socket """        
        socket_stats = [[socket.Q, socket.n] for socket in self.sockets]
        return socket_stats     
    
    def get_mean_reward( self ):
        """ the total reward averaged over the number of time steps """
        return (self.total_reward/self.total_steps)
    
    def get_total_reward_per_timestep( self ):
        """ the cumulative total reward at each timestep of the run """
        return self.total_reward_per_timestep
    
    def get_reward_per_timestep( self ):
        """ the actual reward obtained at each timestep of the run """
        return self.reward_per_timestep
    
    def get_estimates(self):
        """ get the estimate of each socket's reward at each timestep of the run """
        return self.socket_stats[:,:,0]  
    
    def get_number_of_trials(self):
        """ get the number of trials of each socket at each timestep of the run """
        return self.socket_stats[:,:,1]          
                
    def get_socket_percentages( self ):
        """ get the percentage of times each socket was tried over the run """
        return (self.socket_stats[:,:,1][self.total_steps]/self.total_steps)        
    
    def get_time_steps( self ):
        """ get the number of time steps that the test ran for """
        return self.total_steps
    
    def bias_presence_hipo(self, t):
        presence = np.array([i/sum(self.possibles) for i in self.possibles])
        bias_list = []
        for c in range(len(self.sockets)):
          pulled = self.socket_stats[:,:,1][t].copy()
          pulled[c]+=1
          selected_presence = np.array(pulled/(t+1))
          diff = np.subtract(selected_presence, presence)
          #diff = np.divide(diff,presence, out=np.zeros_like(selected_presence), where=presence!=0)
          abs = np.abs(diff)
          bias_list.append(np.sum(abs))
        #return random_argmin(bias_list)
        return bias_list

    def bias_presence_step(self, t):
        presence = np.array([i/sum(self.possibles) for i in self.possibles])
        selected_presence = np.array(self.socket_stats[:,:,1][t]/t)
        diff = np.subtract(selected_presence, presence)
        #diff = np.divide(diff,presence, out=np.zeros_like(selected_presence), where=presence!=0)
        abs = np.abs(diff)
        return np.sum(abs) 

    def final_bias_presence(self):
        presence = np.array([i/sum(self.possibles) for i in self.possibles])
        selected_presence = np.array(self.get_socket_percentages())
        diff = np.subtract(selected_presence, presence)
        abs = np.abs(diff)
        return np.sum(abs) 

    def bias_rate_hipo(self, t):
        bias_list = []
        for c in range(len(self.sockets)):
          pulled = self.socket_stats[:,:,1][t].copy()
          pulled[c]+=1
          curr_percentages = np.array([pulled[i]/self.possibles[i] for i in range(len(self.sockets))])
          overall_percentages = np.mean(curr_percentages)
          diff = np.subtract(overall_percentages,curr_percentages)
          abs = np.abs(diff)
          bias_list.append(np.sum(abs))
        #return random_argmin(bias_list)
        return bias_list

    def bias_rate_step(self, t):
        curr_percentages = np.array([self.get_number_of_trials()[t][i]/self.possibles[i] for i in range(len(self.sockets))])
        overall_percentages = np.mean(curr_percentages)
        diff = np.subtract(overall_percentages,curr_percentages)
        abs = np.abs(diff)
        return np.sum(abs) 

    def final_bias_rate(self):
        final_percentages = np.array([self.get_number_of_trials()[self.total_steps][i]/self.possibles[i] for i in range(len(self.sockets))])
        overall_percentages = self.capping #np.mean(curr_percentages)
        diff = np.subtract(overall_percentages,final_percentages)
        abs = np.abs(diff)
        return np.sum(abs) 
    """
    def select_socket( self, t ):
        #CONSTRAINT A
        curr_percentages = self.socket_stats[:,:,1][t]/t
        mean_percentages = self.capping #np.mean(curr_percentages)
        dif_percentages = np.subtract(mean_percentages,self.socket_stats[:,:,1][t]/t)

        #CONSTRAINT B
        mean_selections = np.mean(self.get_number_of_trials()[t])
        dif_selections = np.subtract(mean_selections,self.get_number_of_trials()[t])

        #DUES CONSTRAINTS, UNA PER PERCENTATGES, L'ALTRE n SELECTIONS
        #dif = dif_percentages #rang 10-30-50
        dif = dif_selections   #rang 0-0.01-0.1

        #print(dif)

        # choose the socket with the current highest mean reward or arbitrarily
        # select a socket in the case of a tie            
        mask = [self.get_number_of_trials()[t][i]<self.possibles[i] for i in range(len(self.sockets))]
        #print(mask)
        available_sockets = [self.sockets[s] for s in range(len(self.sockets)) if (mask[s])]
        available = [i for i in range(len(self.sockets)) if (mask[i])]

        socket_max = random_argmax([self.sockets[socket].sample(t+1)+self.subopt_coef*dif[socket] for socket in available]) 
        #socket_max = random_argmax([self.sockets[socket].sample(t+1) for socket in available]) 

        #print(socket_max)
        socket_index = self.sockets.index(available_sockets[socket_max])
        return socket_index
    """
    def select_socket( self, t ):
        """ Greedy Socket Selection"""
        mask = [self.get_number_of_trials()[t][i]<self.possibles[i] for i in range(len(self.sockets))]
        available_sockets = [self.sockets[s] for s in range(len(self.sockets)) if (mask[s])]
        available = [i for i in range(len(self.sockets)) if (mask[i])]

        socket_max = random_argmax([self.sockets[socket].sample(t+1) for socket in available]) 
        socket_max_index = self.sockets.index(available_sockets[socket_max])

        #print(self.bias_presence_hipo(t))
        socket_min = random_argmin([self.bias_presence_hipo(t)[socket] for socket in available]) 
        #print([self.bias_presence_hipo(t)[socket] for socket in available][socket_min])
        socket_min_index = self.sockets.index(available_sockets[socket_min])
        #print(self.bias_presence_hipo(t)[socket_min_index])

        return_index = random.choices([socket_min_index,socket_max_index],cum_weights=[self.subopt_coef,1])
        #print(socket_min_index,socket_max_index,"->",return_index[0])
        return return_index[0]     
    
    def return_increments(self, decisions_to_consider):
        presence = np.array([i/len(decisions_to_consider) for i in self.possibles])
        selected_presence = np.array(self.get_socket_percentages())
        diff = np.subtract(selected_presence, presence)
        diff = np.divide(diff,presence, out=np.zeros_like(selected_presence), where=presence!=0)
        #print(diff)
        #print(diff.sum())
        return diff
    
    def run( self, decisions_to_consider, max_percent_decisions=1, prints_bool = True):  
        """ perform a single run, over the set of sockets, 
            for the defined number of steps """
        
        # reset the run counters
        self.initialize_run(len(decisions_to_consider))

        self.possibles = [0]*8 #TODO: DEPENDS ON N OF CLUSTER (DECISIONS.UNIQUE() ISNT ENOUGH BECAUSE IT CAN LACK SOME INSTANCES)
        for i in np.unique(np.array(decisions_to_consider)):
          self.possibles[i] = decisions_to_consider.count(i) 
        #print("possibles",self.possibles)

        usos = 0
        self.capping = max_percent_decisions
        usos_maxims = math.floor(max_percent_decisions*len(decisions_to_consider))
        #print(usos_maxims)

        # loop for the specified number of time-steps
        for t in range(1,len(decisions_to_consider)):

            # get information about all sockets at the start of the time step
            self.socket_stats[t] = self.get_socket_stats(t)            
            
            # select a socket
            socket_index = self.select_socket(t)
            #if(decisions_to_consider[t]==socket_index):
            if(self.get_number_of_trials()[t][socket_index]<self.possibles[socket_index]):
              # charge from the chosen socket and update its mean reward value
              self.charge_and_update(socket_index)
              usos+=1
              
              if usos > usos_maxims:
                  #print(usos,usos_maxims)
                  break
        # save the actual number of steps that have been run
        self.total_steps = t    
    
        # get the stats for each socket at the end of the run        
        self.socket_stats[t+1] = self.get_socket_stats(t+1)           
        
        if(prints_bool):
          #print("possibles",self.possibles)
          print("Reward distribution:",[round(s.q,2) for s in self.sockets])
          #print("Intances of each cluster:",self.possibles)
          print("Presence of each in batch (%):",[100*i/len(decisions_to_consider) for i in self.possibles])
          #print("Times selected:",self.get_number_of_trials()[t])
          print("Relative approval rate (%):",[round(100*self.get_number_of_trials()[t][i]/self.possibles[i],2) if self.possibles[i]>0.0 else 0.0 for i in range(len(self.possibles))])
          print("Percentage over total selected (%):",[round(100*p,2) for p in self.get_socket_percentages()])
          print("Total reward:",self.total_reward,"|| and presence bias:",self.final_bias_presence())
          #print("For a total reward:",self.total_reward)
          #print(self.return_increments(decisions_to_consider))

        return self.total_steps, self.total_reward, self.return_increments(decisions_to_consider)
  

In [96]:
decisions = random.choices(range(8),weights=list(instances_bias),k=5000)

In [104]:
sol_test = SocketTesterBatch(PowerSocket, mus, devs, 10)
ts,tr,ti = sol_test.run(decisions, 0.8, True)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [56.76, 23.12, 2.2, 1.0, 8.9, 7.76, 0.14, 0.12]
Relative approval rate (%): [95.31, 100.0, 100.0, 10.0, 2.02, 0.52, 100.0, 100.0]
Percentage over total selected (%): [67.62, 28.9, 2.75, 0.12, 0.22, 0.05, 0.18, 0.15]


In [105]:
sol_test = SocketTesterBatchSOLUTION(PowerSocket, mus, devs, 10, 0)
ts,tr,ti = sol_test.run(decisions, 0.8, True)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [56.76, 23.12, 2.2, 1.0, 8.9, 7.76, 0.14, 0.12]
Relative approval rate (%): [95.95, 100.0, 39.09, 22.0, 11.24, 1.03, 100.0, 100.0]
Percentage over total selected (%): [68.06, 28.89, 1.07, 0.27, 1.25, 0.1, 0.17, 0.15]
Total reward: 375089.44504961284 || and presence bias: 0.34296355911022247


In [106]:
sol_test = SocketTesterBatchSOLUTION(PowerSocket, mus, devs, 10, 0.35)
ts,tr,ti = sol_test.run(decisions, 0.8, True)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [56.76, 23.12, 2.2, 1.0, 8.9, 7.76, 0.14, 0.12]
Relative approval rate (%): [76.07, 100.0, 100.0, 100.0, 84.04, 35.57, 100.0, 100.0]
Percentage over total selected (%): [53.96, 28.89, 2.75, 1.25, 9.35, 3.45, 0.17, 0.15]
Total reward: 371294.64761665964 || and presence bias: 0.14193711572106973


In [109]:
sol_test = SocketTesterBatchSOLUTION(PowerSocket, mus, devs, 10, 0.5)
ts,tr,ti = sol_test.run(decisions, 0.8, True)

Reward distribution: [92.1, 98.48, 88.49, 80.36, 89.97, 73.78, 97.18, 91.52]
Presence of each in batch (%): [56.76, 23.12, 2.2, 1.0, 8.9, 7.76, 0.14, 0.12]
Relative approval rate (%): [70.01, 100.0, 100.0, 100.0, 100.0, 61.6, 100.0, 100.0]
Percentage over total selected (%): [49.66, 28.89, 2.75, 1.25, 11.12, 5.97, 0.17, 0.15]
Total reward: 370182.6904264609 || and presence bias: 0.17742824293926518


In [ ]:
#sol_test = SocketTesterBatchSOLUTION(PowerSocket, list(rewards_bias), 5, 50)
#decisions = random.choices(range(8),weights=list(instances_bias),k=1000)
#ts,tr,ti = sol_test.run(decisions, 0.8, True)

curr_percentages = self.socket_stats[:,:,1][t]/t
mean_percentages = np.mean(curr_percentages)
dif_percentages = np.subtract(mean_percentages,self.socket_stats[:,:,1][t]/t)

mean_selections = np.mean(self.get_number_of_trials()[t])
dif_selections = np.subtract(mean_selections,self.get_number_of_trials()[t])

#DUES CONSTRAINTS, UNA PER PERCENTATGES, L'ALTRE n SELECTIONS
//dif = dif_percentages #rang 10-30-50

dif = dif_selections   #rang 0-0.01-0.1

In [111]:
def nonstatic_sol(iter, rewards_d_o_mu, rewards_d_o_dev, r_coef, instances_d_o, i_coef, sub_opt):
  rewards_d = np.array(rewards_d_o_mu)
  instances_d = np.array(instances_d_o)

  #iter = 1000
  for i in range(iter):
    pbool=False
    if(i%math.ceil(iter/4)==0 or i==iter-1):
        print(i)
        #print(list(instances_distribution))
        pbool=True

    updated_test = SocketTesterBatchSOLUTION(PowerSocket, list(rewards_d), rewards_d_o_dev, 10, sub_opt)
    decisions = random.choices(range(8),weights=list(instances_d),k=5000)
    ts,tr,ti = updated_test.run(decisions, 0.8, pbool)

    increments = ti

    rewards_d = np.add(rewards_d,increments*r_coef*100)
    rewards_d = np.clip(rewards_d,0,100)

    instances_d = np.add(instances_d,increments*i_coef)
    instances_d = np.clip(instances_d,0,None)
    instances_d = instances_d / instances_d.sum()

In [122]:
nonstatic(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [55.5, 23.36, 2.58, 1.24, 9.06, 8.12, 0.1, 0.04]
Relative approval rate (%): [100.0, 86.99, 100.0, 100.0, 1.55, 1.48, 60.0, 100.0]
Percentage over total selected (%): [69.38, 25.4, 3.22, 1.55, 0.18, 0.15, 0.08, 0.05]
25
Reward distribution: [86.17500000000014, 81.94278703167075, 83.13789873702866, 79.68289635308444, 68.56945327357961, 63.880866474102035, 74.76133026695533, 67.24361742424249]
Presence of each in batch (%): [57.82, 25.66, 2.92, 0.76, 6.76, 5.92, 0.12, 0.04]
Relative approval rate (%): [100.0, 71.08, 100.0, 100.0, 0.59, 1.01, 100.0, 100.0]
Percentage over total selected (%): [72.28, 22.8, 3.65, 0.95, 0.05, 0.08, 0.15, 0.05]
50
Reward distribution: [86.80000000000028, 81.38353267442156, 83.7628987370288, 79.76630209503452, 66.1162574752529, 61.42052820659724, 74.91925353241545, 67.19391504329019]
Presence of each in batch (%): [62.34, 25.08, 3.34, 1.08, 4.16, 3.68

In [123]:
nonstatic_sol(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001, 0)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [55.22, 23.56, 2.46, 1.28, 8.94, 8.34, 0.14, 0.06]
Relative approval rate (%): [100.0, 88.46, 100.0, 100.0, 0.45, 0.48, 42.86, 100.0]
Percentage over total selected (%): [69.01, 26.04, 3.07, 1.6, 0.05, 0.05, 0.07, 0.07]
Total reward: 337883.98384872236 || and presence bias: 0.3446509372656836
25
Reward distribution: [86.17421894526352, 81.91149644719317, 83.46421894526353, 79.91355789041629, 68.57872974380561, 63.880788960691284, 74.81540925374706, 67.28743143579177]
Presence of each in batch (%): [57.56, 24.78, 3.04, 1.04, 7.24, 6.04, 0.16, 0.14]
Relative approval rate (%): [100.0, 74.25, 100.0, 59.62, 0.28, 0.99, 100.0, 100.0]
Percentage over total selected (%): [71.93, 22.99, 3.8, 0.77, 0.02, 0.07, 0.2, 0.17]
Total reward: 340953.41777066735 || and presence bias: 0.30436940764808795
50
Reward distribution: [86.79843789052704, 81.3800695479081, 83.84007916879351, 80.04702319

In [124]:
nonstatic_sol(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001, 0.1)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [55.9, 23.76, 1.98, 1.36, 8.22, 8.48, 0.1, 0.2]
Relative approval rate (%): [100.0, 82.66, 100.0, 100.0, 5.84, 4.01, 100.0, 100.0]
Percentage over total selected (%): [69.86, 24.54, 2.47, 1.7, 0.6, 0.42, 0.12, 0.25]
Total reward: 337190.9742989297 || and presence bias: 0.3132551862034491
25
Reward distribution: [86.17421894526352, 81.78337578813091, 83.46421894526353, 80.36694395467462, 68.7659573413739, 64.00541590121946, 75.18463228177905, 67.57558768415572]
Presence of each in batch (%): [58.12, 24.52, 2.7, 1.52, 6.28, 6.0, 0.6, 0.26]
Relative approval rate (%): [100.0, 63.7, 100.0, 100.0, 12.1, 7.0, 100.0, 100.0]
Percentage over total selected (%): [72.63, 19.52, 3.37, 1.9, 0.95, 0.52, 0.75, 0.32]
Total reward: 338886.5369377661 || and presence bias: 0.3158550362409397
50
Reward distribution: [86.79843789052704, 81.13315832411806, 84.08843789052705, 80.90785039473113, 66.5

In [125]:
nonstatic_sol(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001, 0.2)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [54.68, 24.82, 2.46, 1.26, 8.44, 8.06, 0.16, 0.12]
Relative approval rate (%): [100.0, 76.87, 100.0, 100.0, 17.3, 9.68, 100.0, 100.0]
Percentage over total selected (%): [68.33, 23.84, 3.07, 1.57, 1.82, 0.97, 0.2, 0.15]
Total reward: 336844.7463386249 || and presence bias: 0.29328327918020497
25
Reward distribution: [86.17421894526352, 81.65649106754347, 83.46421894526353, 80.5374399257327, 69.01131267053128, 64.16785901786807, 75.25030850341626, 67.82949975469081]
Presence of each in batch (%): [57.24, 23.92, 3.12, 1.56, 6.98, 5.92, 0.52, 0.74]
Relative approval rate (%): [100.0, 63.04, 100.0, 100.0, 17.48, 8.78, 100.0, 100.0]
Percentage over total selected (%): [71.53, 18.85, 3.9, 1.95, 1.52, 0.65, 0.65, 0.92]
Total reward: 337516.1992912485 || and presence bias: 0.31575516120969754
50
Reward distribution: [86.79843789052704, 80.96516055998262, 84.08843789052705, 81.16165887

In [126]:
nonstatic_sol(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001, 0.5)

0
Reward distribution: [85.55, 81.93, 82.84, 79.94, 71.03, 66.35, 74.71, 67.31]
Presence of each in batch (%): [55.76, 24.02, 2.04, 1.16, 8.62, 8.18, 0.12, 0.1]
Relative approval rate (%): [97.85, 94.75, 100.0, 27.59, 1.16, 0.49, 100.0, 60.0]
Percentage over total selected (%): [68.18, 28.44, 2.55, 0.4, 0.12, 0.05, 0.15, 0.07]
Total reward: 337083.4326196114 || and presence bias: 0.3479533116720819
25
Reward distribution: [86.1573197408812, 81.54886847318248, 83.46421894526353, 80.38561271488884, 69.3464890299629, 64.40763350628706, 75.33421894526352, 67.74994218836581]
Presence of each in batch (%): [56.56, 24.68, 3.06, 1.44, 7.5, 5.46, 0.8, 0.5]
Relative approval rate (%): [100.0, 50.73, 100.0, 100.0, 44.27, 32.97, 100.0, 100.0]
Percentage over total selected (%): [70.68, 15.65, 3.82, 1.8, 4.15, 2.25, 1.0, 0.62]
Total reward: 335454.602244362 || and presence bias: 0.31166028492876774
50
Reward distribution: [86.74543639794115, 80.97827231621457, 83.96070769094274, 80.8077303027388, 6

In [1]:
nonstatic_sol(100, rewards_bias, [10.0]*8, 0.001, instances_bias, 0.001, 1)

NameError: ignored

In [ ]:
variances = [10]*8
##(5000 per iter)
nonstatic_sol(1000, rewards_bias, 0.02, instances_bias, 0.02, 0.1, variances)

In [ ]:
variances = [10]*8
nonstatic_sol(1000, rewards_bias, 0.04, instances_bias, 0.04, 100, variances)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
import pandas as pd
import numpy as np

from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)

unpickled_df = pd.read_csv("/content/drive/MyDrive/TFG/hmda/state_AL-GA_actions_taken_1-3.csv")
print(unpickled_df.derived_sex.unique(),'\n',
unpickled_df.derived_race.unique(),'\n',
unpickled_df.derived_ethnicity.unique(),'\n',
unpickled_df.action_taken.unique())

subsample = unpickled_df                    

print(len(subsample.loc[subsample["action_taken"]==1]),len(subsample.loc[subsample["action_taken"]==3]))
subsample = subsample.loc[subsample["action_taken"].isin([1,3])]
subsample["action_taken"] = subsample["action_taken"].replace([3],[0])    
print("action:",subsample["action_taken"].unique())
print(len(subsample.loc[subsample["action_taken"]==1]),len(subsample.loc[subsample["action_taken"]==0]))

print("race pre:",subsample["derived_race"].unique())
subsample = subsample.loc[subsample["derived_race"].isin(["White","Black or African American"])]
subsample["derived_race"] = subsample["derived_race"].replace(["White","Black or African American"],[0,1])
print("race:",subsample["derived_race"].unique())

print("ethnicity pre:",subsample["derived_ethnicity"].unique())
subsample = subsample.loc[subsample["derived_ethnicity"].isin(['Not Hispanic or Latino','Hispanic or Latino'])]
subsample["derived_ethnicity"] = subsample["derived_ethnicity"].replace(['Not Hispanic or Latino','Hispanic or Latino'],[0,1])
print("ethnicity:",subsample["derived_ethnicity"].unique())

print("sex pre",subsample["derived_sex"].unique())
subsample = subsample.loc[subsample["derived_sex"].isin(['Male','Female'])]
subsample["derived_sex"] = subsample["derived_sex"].replace(['Male','Female'],[0,1])
print("sex:",subsample["derived_sex"].unique())


subsample = subsample.loc[subsample["interest_rate"]!='Exempt']
subsample["interest_rate"] = pd.to_numeric(subsample["interest_rate"] ) 

#print(len(subsample.loc[subsample["loan_to_value_ratio"]=='Exempt']))
subsample = subsample.loc[subsample["loan_to_value_ratio"]!='Exempt']
#print(len(subsample.loc[subsample["loan_to_value_ratio"]=='Exempt']))
#print(subsample["loan_to_value_ratio"])
subsample["loan_to_value_ratio"] = pd.to_numeric(subsample["loan_to_value_ratio"] )  
#print(subsample["loan_to_value_ratio"].mean())

subsample = subsample.loc[subsample["property_value"]!='Exempt']
subsample["property_value"] = pd.to_numeric(subsample["property_value"] ) 

print(len(subsample.loc[subsample["action_taken"]==1]),len(subsample.loc[subsample["action_taken"]==0]))

print(len(subsample))
#subsample = subsample[["loan_type", "property_type", "loan_purpose", "loan_amount_000s",
#                            "action_taken","applicant_ethnicity","applicant_race_1","applicant_sex",
#                           "applicant_income_000s"]]
print(len(subsample.loc[subsample["action_taken"]==1]),len(subsample.loc[subsample["action_taken"]==0]))
print(len(subsample),len(subsample.columns),subsample.isnull().values.sum())

#thr=0.25
#for col in subsample.columns:
#  if (subsample[col].isnull().sum()>thr*len(subsample)):
    #subsample.drop(columns=[col])
#    print(col)

#subsample = subsample.dropna(axis=1,thresh=thr*len(subsample))
#subsample = subsample.dropna(axis=0)

# Iterate over each column of cc_apps
for col in subsample.columns:
    # Check if the column is of object type
    if subsample[col].dtypes == 'object':
        # Impute with the most frequent value
        subsample = subsample.fillna(subsample[col].value_counts().index[0])


print(len(subsample.loc[subsample["action_taken"]==1]),len(subsample.loc[subsample["action_taken"]==0]))

# Count the number of NaNs in the dataset and print the counts to verify
print(len(subsample),len(subsample.columns),subsample.isnull().values.sum())
cols = [f_ for f_ in subsample.columns if subsample[f_].dtype != 'object']
features = cols

list_to_remove = ['action_taken','purchaser_type', 'activity_year',
                  'denial_reason_1','denial_reason_2','denial_reason_3','sequence_number','application_date_indicator']

features = list(set(cols).difference(set(list_to_remove)))

X = subsample[features]
y = subsample['action_taken']

# Import train_test_split
from sklearn.model_selection import train_test_split
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                y,
                                test_size=0.3,
                                random_state=2)

X_test1, X_test2, y_test1, y_test2 = train_test_split(X_test,
                                y_test,
                                test_size=0.33,
                                random_state=2)

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)
rescaledX_test1 = scaler.fit_transform(X_test1)
rescaledX_test2 = scaler.fit_transform(X_test2)

logreg = LogisticRegression(solver="liblinear")

scores_dict = cross_validate(logreg,rescaledX_train,y_train,cv=5, return_estimator=True)
best = scores_dict["estimator"][0] 


y_pred = best.predict(rescaledX_test1)
from collections import Counter
counterpred = Counter(y_pred)
countertest = Counter(y_test1)
counterall = Counter(y)
print(counterpred, countertest, counterall)
print("Accuracy of logistic regression classifier: ", best.score(rescaledX_test1, y_test1))


y_pred = best.predict(rescaledX_test2)
counterpred = Counter(y_pred)
countertest = Counter(y_test2)
counterall = Counter(y)
print(counterpred, countertest, counterall)
print("Accuracy of logistic regression classifier: ", best.score(rescaledX_test2, y_test2))

In [ ]:
import math
def partitions(n_partitions, X, y):
  X_partitions = []
  y_partitions = []
  size_partitions = math.floor(len(X)/n_partitions)
  for p in range(n_partitions):
    X_partitions.append(X[p*size_partitions:(p+1)*size_partitions])  
    y_partitions.append(y[p*size_partitions:(p+1)*size_partitions])
  print(len(X), [sum(len(item) for item in X_partitions)], len(X_partitions[0]))
  return X_partitions, y_partitions

In [ ]:
def create_df(x,xcol,y,ycol):
  auxnparray = x.copy()
  newdf = pd.DataFrame(auxnparray, columns = xcol)
  newdf[ycol] = y.tolist()
  return newdf

In [ ]:
def preproc(df):
  subgroups = []
  instances = []
  sensitive = ["derived_race","derived_ethnicity","derived_sex"]
  R = ["W","B"]
  E = ["NH","H"]
  S = ["M","F"]
  for option in [0,1]:
    for option2 in [0,1]:
      for option3 in [0,1]:
          subgroups.append(df.loc[((df[sensitive[0]]==option) & (df[sensitive[1]]==option2) & (df[sensitive[2]]==option3))])
          instances.append(len(subgroups[-1]))
  return subgroups, instances

In [ ]:
from scipy.stats import norm

df = create_df(X,features,y,"action_taken")

mean1, std1 = norm.fit(df.loc[df["action_taken"]==1.0]["loan_amount"])

mean0, std0 = norm.fit(df.loc[df["action_taken"]==0.0]["loan_amount"])

print(mean1, std1)
print(mean0, std0)
print()
sg, ic = preproc(df)
for s in sg:
  approval_rate = len(s.loc[s["action_taken"]==1.0])/len(s)
  print(approval_rate)
  mu, std = norm.fit(s["loan_amount"])
  print(mu,std)

In [ ]:
class PowerSocketReal:
    """ the base power socket class """
    
    def __init__(self, q, cl, var=1):                
        self.q = q        # the true reward value 
        self.confidence_level = cl          
        self.var = var   
        self.initialize() # reset the socket
        
    def initialize(self):
        self.Q = 0   # the estimate of this socket's reward value                
        self.n = 0   # the number of times this socket has been tried        
    
    def charge(self):
        """ return a random amount of charge """
        
        # the reward is a guassian distribution with unit variance around the true
        # value 'q'
        # value = self.var * np.random.randn() + self.q      

        #value = self.q.iloc[self.n]["action_taken"]

        #value = self.q["action_taken"].sample(n=1).iloc[0]

        sample = self.q.sample(n=1)
        if (sample["action_taken"].iloc[0]==1.0):
          value = sample["loan_amount"].iloc[0]
        else:
          value = 0.0

        # never allow a charge less than 0 to be returned        
        return 0 if value < 0 else value
               
    def update(self,R):
        """ update this socket after it has returned reward value 'R' """     
    
        # increment the number of times this socket has been tried
        self.n += 1

        # the new estimate of the mean is calculated from the old estimate
        self.Q = (1 - 1.0/self.n) * self.Q + (1.0/self.n) * R

    def uncertainty(self, t): 
        """ calculate the uncertainty in the estimate of this socket's mean """
        if self.n == 0: return float('inf')                         
        return self.confidence_level * (np.sqrt(np.log(t) / self.n))         
        
    def sample(self,t):
        """ the UCB reward is the estimate of the mean reward plus its uncertainty """
        return self.Q + self.uncertainty(t) 

In [ ]:
class SocketTesterBatchReal():
    """ create and test a set of sockets over a single test run """

    def __init__(self, socket, socket_order, confidence_level):  
        
        # create supplied socket type with a mean value defined by socket order 
        self.sockets = [socket(q, confidence_level) for q in socket_order]     
        
        # set the number of sockets equal to the number created
        self.number_of_sockets = len(self.sockets)

        self.number_of_stats = 2                 
            
    def initialize_run(self, number_of_steps):
        """ reset counters at the start of a run """
        
        # save the number of steps over which the run will take place
        self.number_of_steps = number_of_steps
        
        # reset the actual number of steps that the test ran for
        self.total_steps = 0
        
        # monitor the total reward obtained over the run
        self.total_reward = 0
        
        # the current total reward at each timestep of the run
        self.total_reward_per_timestep = []
        
        # the actual reward obtained at each timestep
        self.reward_per_timestep = []
           
        # stats for each time-step
        # - by default records: estimate, number of trials
        self.socket_stats = np.zeros(shape=(number_of_steps+1, 
                                            self.number_of_sockets, 
                                            self.number_of_stats))
        
        # ensure that all sockets are re-initialized
        for socket in self.sockets: socket.initialize()
            
                                
    def charge_and_update(self,socket_index):
        """ charge from & update the specified socket and associated parameters """
        
        # charge from the chosen socket and update its mean reward value
        reward = self.sockets[socket_index].charge()
        self.sockets[socket_index].update(reward)

        # update the total reward
        self.total_reward += reward   
        
        # store the current total reward at this timestep
        self.total_reward_per_timestep.append(self.total_reward)
        
        # store the reward obtained at this timestep
        self.reward_per_timestep.append(reward)        
        
        
    def get_socket_stats( self, t ):
        """ get the current information from each socket """        
        socket_stats = [[socket.Q, socket.n] for socket in self.sockets]
        return socket_stats     
    
    def get_mean_reward( self ):
        """ the total reward averaged over the number of time steps """
        return (self.total_reward/self.total_steps)
    
    def get_total_reward_per_timestep( self ):
        """ the cumulative total reward at each timestep of the run """
        return self.total_reward_per_timestep
    
    def get_reward_per_timestep( self ):
        """ the actual reward obtained at each timestep of the run """
        return self.reward_per_timestep
    
    def get_estimates(self):
        """ get the estimate of each socket's reward at each timestep of the run """
        return self.socket_stats[:,:,0]  
    
    def get_number_of_trials(self):
        """ get the number of trials of each socket at each timestep of the run """
        return self.socket_stats[:,:,1]          
                
    def get_socket_percentages( self ):
        """ get the percentage of times each socket was tried over the run """
        return (self.socket_stats[:,:,1][self.total_steps]/self.total_steps)        
    
    def get_time_steps( self ):
        """ get the number of time steps that the test ran for """
        return self.total_steps
    
    def select_socket( self, t ):
        """ Greedy Socket Selection"""
        
        # choose the socket with the current highest mean reward or arbitrarily
        # select a socket in the case of a tie            
        mask = [self.get_number_of_trials()[t][i]<self.possibles[i] for i in range(len(self.sockets))]
        #print(mask)
        available = [self.sockets[i] for i in range(len(self.sockets)) if (mask[i])]
        #print([socket.sample(t+1) for socket in available])
        socket_max = random_argmax([socket.sample(t+1) for socket in available]) 
        #print(socket_max)
        socket_index = self.sockets.index(available[socket_max])
        return socket_index     
    
    def return_increments(self, decisions_to_consider):
        presence = np.array([i/len(decisions_to_consider) for i in self.possibles])
        selected_presence = np.array(self.get_socket_percentages())
        diff = np.subtract(selected_presence, presence)
        #print(diff)
        #print(diff.sum())
        return diff
    
    def run( self, decisions_to_consider, class_count, max_percent_decisions=1, prints_bool = True):  
        """ perform a single run, over the set of sockets, 
            for the defined number of steps """
        
        for s in range(len(self.sockets)):
          self.sockets[s].q = decisions_to_consider[s]     

        # reset the run counters
        self.initialize_run(sum(class_count))

        self.possibles = class_count
        print(self.possibles)

        usos = 0
        usos_maxims = math.floor(max_percent_decisions*sum(class_count))
        #print(usos_maxims)

        # loop for the specified number of time-steps
        for t in range(sum(class_count)):

            # get information about all sockets at the start of the time step
            self.socket_stats[t] = self.get_socket_stats(t)            
            
            # select a socket
            socket_index = self.select_socket(t)
            #if(decisions_to_consider[t]==socket_index):
            if(self.get_number_of_trials()[t][socket_index]<self.possibles[socket_index]):
              # charge from the chosen socket and update its mean reward value
              self.charge_and_update(socket_index)
              usos+=1
              
              if usos > usos_maxims:
                  #print(usos,usos_maxims)
                  break
        # save the actual number of steps that have been run
        self.total_steps = t    
    
        # get the stats for each socket at the end of the run        
        self.socket_stats[t+1] = self.get_socket_stats(t+1)           
        
        if(prints_bool):
          #print("Reward distribution:",[s.q for s in self.sockets])
          #print("Intances of each cluster:",self.possibles)
          print("Presence of each in batch (%):",[100*i/sum(class_count) for i in self.possibles])
          #print("Times selected:",self.get_number_of_trials()[t])
          print("Relative approval rate (%):",[round(100*self.get_number_of_trials()[t][i]/self.possibles[i],2) if self.possibles[i]>0.0 else 0.0 for i in range(len(self.possibles))])
          print("Percentage over total selected (%):",[round(100*p,2) for p in self.get_socket_percentages()])
          #print("For a total reward:",self.total_reward)


        return self.total_steps, self.total_reward, self.return_increments(decisions_to_consider)

In [ ]:
class SocketTesterBatchRealBudget():
    """ create and test a set of sockets over a single test run """

    def __init__(self, socket, socket_order, confidence_level):  
        
        # create supplied socket type with a mean value defined by socket order 
        self.sockets = [socket(q, confidence_level) for q in socket_order]     
        
        # set the number of sockets equal to the number created
        self.number_of_sockets = len(self.sockets)

        self.number_of_stats = 2                 
            
    def initialize_run(self, number_of_steps):
        """ reset counters at the start of a run """
        
        # save the number of steps over which the run will take place
        self.number_of_steps = number_of_steps
        
        # reset the actual number of steps that the test ran for
        self.total_steps = 0
        
        # monitor the total reward obtained over the run
        self.total_reward = 0
        
        # the current total reward at each timestep of the run
        self.total_reward_per_timestep = []
        
        # the actual reward obtained at each timestep
        self.reward_per_timestep = []
           
        # stats for each time-step
        # - by default records: estimate, number of trials
        self.socket_stats = np.zeros(shape=(number_of_steps+1, 
                                            self.number_of_sockets, 
                                            self.number_of_stats))
        
        # ensure that all sockets are re-initialized
        for socket in self.sockets: socket.initialize()
            
                                
    def charge_and_update(self,socket_index):
        """ charge from & update the specified socket and associated parameters """
        
        # charge from the chosen socket and update its mean reward value
        reward = self.sockets[socket_index].charge()
        self.sockets[socket_index].update(reward)

        # update the total reward
        self.total_reward += reward  
        self.budget -= reward 
        
        # store the current total reward at this timestep
        self.total_reward_per_timestep.append(self.total_reward)
        
        # store the reward obtained at this timestep
        self.reward_per_timestep.append(reward)        
        
        
    def get_socket_stats( self, t ):
        """ get the current information from each socket """        
        socket_stats = [[socket.Q, socket.n] for socket in self.sockets]
        return socket_stats     
    
    def get_mean_reward( self ):
        """ the total reward averaged over the number of time steps """
        return (self.total_reward/self.total_steps)
    
    def get_total_reward_per_timestep( self ):
        """ the cumulative total reward at each timestep of the run """
        return self.total_reward_per_timestep
    
    def get_reward_per_timestep( self ):
        """ the actual reward obtained at each timestep of the run """
        return self.reward_per_timestep
    
    def get_estimates(self):
        """ get the estimate of each socket's reward at each timestep of the run """
        return self.socket_stats[:,:,0]  
    
    def get_number_of_trials(self):
        """ get the number of trials of each socket at each timestep of the run """
        return self.socket_stats[:,:,1]          
                
    def get_socket_percentages( self ):
        """ get the percentage of times each socket was tried over the run """
        return (self.socket_stats[:,:,1][self.total_steps]/self.total_steps)        
    
    def get_time_steps( self ):
        """ get the number of time steps that the test ran for """
        return self.total_steps
    
    def select_socket( self, t ):
        """ Greedy Socket Selection"""
        
        # choose the socket with the current highest mean reward or arbitrarily
        # select a socket in the case of a tie            
        mask = [self.get_number_of_trials()[t][i]<self.possibles[i] for i in range(len(self.sockets))]
        #print(mask)
        available = [self.sockets[i] for i in range(len(self.sockets)) if (mask[i])]
        #print([socket.sample(t+1) for socket in available])
        socket_max = random_argmax([socket.sample(t+1) for socket in available]) 
        #print(socket_max)
        socket_index = self.sockets.index(available[socket_max])
        return socket_index     
    
    def return_increments(self, decisions_to_consider):
        presence = np.array([i/len(decisions_to_consider) for i in self.possibles])
        selected_presence = np.array(self.get_socket_percentages())
        diff = np.subtract(selected_presence, presence)
        #print(diff)
        #print(diff.sum())
        return diff
    
    def run( self, decisions_to_consider, class_count, budget, prints_bool = True):  
        """ perform a single run, over the set of sockets, 
            for the defined number of steps """
        
        for s in range(len(self.sockets)):
          self.sockets[s].q = decisions_to_consider[s]     

        # reset the run counters
        self.initialize_run(sum(class_count))

        self.possibles = class_count
        print(self.possibles)

        usos = 0
        self.budget = budget
        #print(usos_maxims)

        # loop for the specified number of time-steps
        for t in range(sum(class_count)):

            # get information about all sockets at the start of the time step
            self.socket_stats[t] = self.get_socket_stats(t)            
            
            # select a socket
            socket_index = self.select_socket(t)
            #if(decisions_to_consider[t]==socket_index):
            if(self.get_number_of_trials()[t][socket_index]<self.possibles[socket_index]):
              # charge from the chosen socket and update its mean reward value
              self.charge_and_update(socket_index)
              usos+=1
            if self.budget-max(self.get_estimates()[t]) < 0:
              break
        # save the actual number of steps that have been run
        self.total_steps = t    
    
        # get the stats for each socket at the end of the run        
        self.socket_stats[t+1] = self.get_socket_stats(t+1)           
        
        if(prints_bool):
          #print("Reward distribution:",[s.q for s in self.sockets])
          #print("Intances of each cluster:",self.possibles)
          print("Presence of each in batch (%):",[100*i/sum(class_count) for i in self.possibles])
          #print("Times selected:",self.get_number_of_trials()[t])
          print("Relative approval rate (%):",[round(100*self.get_number_of_trials()[t][i]/self.possibles[i],2) if self.possibles[i]>0.0 else 0.0 for i in range(len(self.possibles))])
          print("Percentage over total selected (%):",[round(100*p,2) for p in self.get_socket_percentages()])
          #print("For a total reward:",self.total_reward)


        return self.total_steps, self.total_reward, self.return_increments(decisions_to_consider)

In [ ]:
real_test = SocketTesterBatchReal(PowerSocketReal, [[0]]*8, 5)
print(real_test.sockets)


X_p, y_p = partitions(4,X,y)
for b in range(4):
  batch = create_df(X_p[b],features,y_p[b],"action_taken")
  sg, ic = preproc(batch)
  ts,tr,ti = real_test.run(sg, ic, 0.8, True)

In [ ]:
real_test = SocketTesterBatchRealBudget(PowerSocketReal, [[0]]*8, 5)
print(real_test.sockets)


X_p, y_p = partitions(10,X,y)
for b in range(10):
  batch = create_df(X_p[b],features,y_p[b],"action_taken")
  budget = batch["loan_amount"].sum()
  sg, ic = preproc(batch)
  ts,tr,ti = real_test.run(sg, ic, 0.8*budget, True)

In [ ]:
real_test = SocketTesterBatchRealBudget(PowerSocketReal, [[0]]*8, 5)
print(real_test.sockets)


X_p, y_p = partitions(1,X,y)
for b in range(1):
  batch = create_df(X_p[b],features,y_p[b],"action_taken")
  #budget = batch["loan_amount"].sum()
  budget=0
  sg, ic = preproc(batch)
  min_amount = min(ic)
  for s in range(len(sg)):
    sg[s] = sg[s].sample(n=min_amount)
    budget += sg[s]["loan_amount"].sum()
  ts,tr,ti = real_test.run(sg, [min_amount]*8, 0.8*budget, True)